# Objectives 
- 고객의 대출 등급에 따른 분류


train.csv [파일] 
고객 관련 금융 정보
ID : 대출 고객의 고유 ID
대출등급 : 예측 목표


test.csv [파일]
고객 관련 금융 정보
ID : 대출 고객의 고유 ID
대출등급이 존재하지 않음


sample_submission.csv [파일] - 제출 양식
ID : 대출 고객의 고유 ID
대출등급 : test.csv에서 제공된 고객의 대출등급을 예측하여 기입


## Import Functions

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import random
import os
import matplotlib.pyplot as plt
import seaborn as sns

## Load Train Data

In [ ]:
# 학습용 데이터 불러오기

train_path = 'D:\Code\[중]고객 대출 등급 분류 해커톤/train.csv'

df_train = pd.read_csv(train_path)

print(f'학습용 데이터 : \n{df_train}')

df_train.isnull().sum()

# 누락 데이터 없음

In [ ]:
df_train.info()

In [ ]:
# 정답, 레이블 컬럼 : 대출등급
label = df_train.loc[:,'대출등급']

cleaned_df=df_train.drop(columns=['대출등급'])

print('*'*80)

print(label)

print('*'*80)

print(cleaned_df)

## Outlier(이상치) 확인

In [ ]:
cleaned_df.columns

In [14]:
cleaned_df.loc[:,['ID', '대출금액', '대출기간', '근로기간', '주택소유상태', '연간소득', '부채_대비_소득_비율', '총계좌수',
       '대출목적', '최근_2년간_연체_횟수', '총상환원금', '총상환이자', '총연체금액', '연체계좌수']].plot(kind='bar', rot=45)
plt.show()

## Load Test Data

In [ ]:
test_path = 'D:\Code\[중]고객 대출 등급 분류 해커톤/test.csv'

df_test = pd.read_csv(test_path)

print(f'학습용 데이터 : \n{df_test}')